In [ ]:
def recommend_movies(tv_series_title, user_preferences, top_n=10):
    # Layered filtering
    filtered_tv_df = tv_df[
        (tv_df['age_appropriate']) &
        (tv_df['language'] == user_preferences['language']) &
        (tv_df['popularity'] >= user_preferences['popularity_threshold'])
    ]
    filtered_tv_df = filter_by_genre(filtered_tv_df, user_preferences['genres'])
    
    filtered_movies_df = movies_df[
        (movies_df['age_appropriate']) &
        (movies_df['language'] == user_preferences['language']) &
        (movies_df['popularity'] >= user_preferences['popularity_threshold'])
    ]
    filtered_movies_df = filter_by_genre(filtered_movies_df, user_preferences['genres'])
    
    # LLM-based similarity
    target_overview = tv_df[tv_df['name'] == tv_series_title]['overview'].values[0]
    movie_overviews = filtered_movies_df['overview'].tolist()
    similar_overviews = get_similar_overview(target_overview, movie_overviews)
    
    # Get top N recommendations
    recommendations = filtered_movies_df[filtered_movies_df['overview'].isin(similar_overviews)].head(top_n)
    return recommendations
